# Assignment 5
    by P.S.Karthik, AM.EN.U4CSE18509

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from numpy import array
from numpy import asarray
from numpy import zeros
from numpy import array
import keras
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
nltk.download('stopwords')

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91891\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### 1. Download MoviewRevies/RestaurantReviews dataset from Kaggle

Dataset: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [2]:
df = pd.read_csv('Data/IMDB Dataset.csv')

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
def preprocess_text(sen):
    sentence = remove_tags(sen)
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [5]:
X = []
sentences = list(df['review'])
for sen in sentences:
    X.append(preprocess_text(sen))

In [6]:
y = df['sentiment']

y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y)))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

### 2. Apply basic vectorization if the data in text format

In [8]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [10]:
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [11]:
embeddings_dictionary = dict()
glove_file = open('Data/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [12]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

### 3. Apply classifier based on RNN-LSTM

In [13]:
def build_model():
    model = Sequential()
    embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
    model.add(embedding_layer)
    model.add(LSTM(128))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    return model

In [14]:
model = build_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          9254700   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 9,372,077
Trainable params: 117,377
Non-trainable params: 9,254,700
_________________________________________________________________
None


In [15]:
history = model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

score = model.evaluate(X_test, y_test, verbose=1)

Train on 32000 samples, validate on 8000 samples
Epoch 1/10
32000/32000 [==============================] - 85s 3ms/step - loss: 0.5397 - acc: 0.7234 - val_loss: 0.5122 - val_acc: 0.7420
Epoch 2/10
32000/32000 [==============================] - 91s 3ms/step - loss: 0.4440 - acc: 0.7951 - val_loss: 0.4030 - val_acc: 0.8180
Epoch 3/10
32000/32000 [==============================] - 94s 3ms/step - loss: 0.3988 - acc: 0.8200 - val_loss: 0.3908 - val_acc: 0.8244
Epoch 4/10
32000/32000 [==============================] - 97s 3ms/step - loss: 0.3689 - acc: 0.8353 - val_loss: 0.3854 - val_acc: 0.8264
Epoch 5/10
32000/32000 [==============================] - 101s 3ms/step - loss: 0.3487 - acc: 0.8475 - val_loss: 0.3536 - val_acc: 0.8403
Epoch 6/10
32000/32000 [==============================] - 101s 3ms/step - loss: 0.3324 - acc: 0.8525 - val_loss: 0.3417 - val_acc: 0.8497
Epoch 7/10
32000/32000 [==============================] - 102s 3ms/step - loss: 0.3093 - acc: 0.8674 - val_loss: 0.3317 - val_a

In [16]:
score

[0.3360518484950066, 0.8575000166893005]

Test score and Accuracy on test data of kaggle dataset

In [17]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 0.3360518484950066
Test Accuracy: 0.8575000166893005


### 4. Check the Accuracy with imdb dataset.

Checking accuracy of the model on the keras imdb dataset

In [18]:
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(
    num_words=5000
)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

25000 Training sequences
25000 Validation sequences


In [19]:
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)

In [26]:
score = model.evaluate(x_train, y_train, verbose=1)

25000/25000 [==============================] - 17s 685us/step


In [27]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 1.0568358856201172
Test Accuracy: 0.510919988155365
